The purpose of this notebook is to develop MKID RFDC, Dac Replay, and ADC-Capture Drivers and test the ZCU111 RF system.

# Imports

In [1]:
import pynq
from pynq import PL
import xrfclk
import xrfdc
from pynq import DefaultIP

# Drivers

## DAC Replay

In [2]:
class DACReplay(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)

    bindto = ['mazinlab:mkidgen3:dac_table_axim:1.32']

    @property
    def replay_length(self):
        return self.register_map.replay_length

    @replay_length.setter
    def replay_length(self, rlen):
        self.register_map.replay_length = int(rlen)
    
    def start(self):
        """Start the core.
        
        """
        self.register_map.run = 1
        return

## ADC Capture

In [3]:
class ADCCapture(DefaultIP):
    def __init__(self, description):
        super().__init__(description=description)

    bindto = ['mazinlab:mkidgen3:adc_capture:1.33']

    @property
    def capturesize(self):
        return self.register_map.capturesize

    @capturesize.setter
    def capturesize(self, capsize):
        self.register_map.capturesize = int(capsize)
        
    @property
    def q_addr(self):
        return self.register_map.iqout_1

    @capturesize.setter
    def q_addr(self, q_addr):
        self.register_map.iqout_1 = q_addr
    
    @property
    def i_addr(self):
        return self.register_map.iqout_2

    @capturesize.setter
    def i_addr(self, i_addr):
        self.register_map.iqout_2 = i_addr
    
    def start(self):
        """Run core bring-up sequence.
        
        """
        self.register_map.CTRL.AP_START = 1
        return

## MKID RFDC

In [4]:
# driver exists (see xrfdc)
# MKIDs need new class interiting from existing??

# Load the Overlay

In [5]:
ol = pynq.Overlay('clkw_no_lock.bit', ignore_version=True)
ol.download()
print(f"PL Bitfile: {PL.bitfile_name}\nPL Timestamp: {PL.timestamp}\n"
      f"Overlay timestamp: {ol.timestamp}  Loaded: {ol.is_loaded()}")

PL Bitfile: /home/xilinx/jupyter_notebooks/adc-to-xrt-mig-snap/clkw_no_lock.bit
PL Timestamp: 2021/9/8 22:19:0 +152164
Overlay timestamp: 2021/9/8 22:19:0 +152164  Loaded: True


In [6]:
# Necessary for PL DRAM XRT Allocation
xrt_ol = pynq.Overlay('mig_modified_ip_layout_mem_topology.xclbin', device=pynq.Device.devices[1])
xrt_ol.download()

## Bind Drivers

In [7]:
rfdc=ol.usp_rf_data_converter_0
ddr=ol.ddr4_1
adcsnap=ol.adc_capture_1
dacreplay=ol.dac_table_axim_0

## Allocate a Chunk of PLDRAM for capture

In [8]:
pl_buf = pynq.allocate((1024,), 'u4', target=xrt_ol.MIG0)

## Setup RFDC

In [9]:
xrfclk.set_all_ref_clks(409.6)
# new driver: xrfclk.set_ref_clks(409.6)

In [10]:
adc_tile = rfdc.adc_tiles[1]
adc0 = adc_tile.blocks[0]
adc1 = adc_tile.blocks[1]

In [11]:
adc0.MixerSettings

{'CoarseMixFreq': 16,
 'EventSource': 2,
 'FineMixerScale': 0,
 'Freq': 0.0,
 'MixerMode': 4,
 'MixerType': 1,
 'PhaseOffset': 0.0}

In [12]:
adc_tile.DynamicPLLConfig(1, 409.6, 1228.8)
adc0.NyquistZone = 1
adc0.MixerSettings = {
    'CoarseMixFreq'  : xrfdc.COARSE_MIX_BYPASS,
    'EventSource'    : xrfdc.EVNT_SRC_TILE,
    'FineMixerScale' : xrfdc.MIXER_SCALE_1P0,
    'Freq'           : 64,
    'MixerMode'      : xrfdc.MIXER_MODE_R2C,
    'MixerType'      : xrfdc.MIXER_TYPE_FINE,
    'PhaseOffset'    : 0.0
}
adc0.UpdateEvent(xrfdc.EVENT_MIXER)
adc_tile.SetupFIFO(True)

## Setup DAC Replay

In [13]:
dacreplay.replay_length = 0

## Setup ADC Capture

In [14]:
adcsnap.capturesize = 28 # 27 to 32
adcsnap.q_adrr = (pl_buf.device_address) & (0xffffffff) # set addresses
adcsnap.i_addr = (pl_buf.device_address) >> 32 # set addresses

In [15]:
n_cap = 40000000*3
adc_mb=n_cap*32/1024**2
adc_mbps=32*512
adc_eta=n_cap/512e3
print(f"Will capture {adc_mb:.0f} MB of data @ {adc_mbps} MBps. ETA {adc_eta:.0f} ms")

#adc0.capture(n_cap, start=False, addr=0*2**31/2**12)

Will capture 3662 MB of data @ 16384 MBps. ETA 234 ms
